In [1]:
import pandas as pd
import numpy as np

# Data Visualization Libraries
import seaborn as sns
import matplotlib.pyplot as plt

# Machine Learning Libraries
from sklearn.preprocessing import  StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score ,precision_score, recall_score, f1_score
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.metrics import  classification_report, confusion_matrix

# Machine Learning Models
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier, AdaBoostClassifier
from sklearn.svm import SVC

In [2]:
train_file = "Insurance_Train.csv"
test_file = "Insurance_Test.csv"
train = pd.read_csv(train_file)

test = pd.read_csv(test_file)
train.head()

,id,age,job,marital,education,balance,housing,loan,contact,month,duration,campaign,pdays,previous,poutcome,y
0,98749,32,1,1,2,64,1,0,0,5,202,2,-1,0,0,0
1,19550,45,2,2,2,534,0,0,1,8,104,6,-1,0,0,0
2,75084,45,10,2,2,1477,1,0,1,11,75,1,132,1,1,0
3,65715,39,10,2,3,14,0,0,1,1,114,2,-1,0,0,0
4,41412,49,2,1,0,2222,0,0,0,6,114,2,-1,0,0,0


In [7]:
str_list = [] 
num_list = []
train = train[["age", "job", "housing", "contact", "month", "duration","y"]]
test = test[["age", "job", "housing", "contact", "month", "duration"]]


for colname, colvalue in train.iteritems():
    if type(colvalue.iloc[1]) == str:
        str_list.append(colname)
    else:
        num_list.append(colname)

num_list.remove("y")

train = pd.get_dummies(train, columns=str_list)
test = pd.get_dummies(test, columns=str_list)


In [8]:
X = train.drop(["y"], axis=1)
y = train["y"]

# Create scaler object
scaler = StandardScaler()

# Fit scaler on selected columns
scaler.fit(X[num_list])

# Transform selected columns with scaler
X[num_list] = scaler.transform(X[num_list])

x_train, x_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=10)

In [9]:

models = {
    'Logistic Regression': LogisticRegression(),
    'Decision Tree': DecisionTreeClassifier(),
    'Random Forest': RandomForestClassifier(),
    'SVM': SVC(),
    'GradientBoostingClassifier':GradientBoostingClassifier(),
    'AdaBoostClassifier':AdaBoostClassifier()
    
}

# Train and evaluate each model using cross-validation
for name, model in models.items():
    scores = cross_val_score(model, x_train, y_train, cv=5, scoring='accuracy')
    print(f"{name} accuracy: {scores.mean():.3f} +/- {scores.std():.3f}")


'\nmodels = {\n    \'Logistic Regression\': LogisticRegression(),\n    \'Decision Tree\': DecisionTreeClassifier(),\n    \'Random Forest\': RandomForestClassifier(),\n    \'SVM\': SVC(),\n    \'GradientBoostingClassifier\':GradientBoostingClassifier(),\n    \'AdaBoostClassifier\':AdaBoostClassifier()\n    \n}\n\n# Train and evaluate each model using cross-validation\nfor name, model in models.items():\n    scores = cross_val_score(model, x_train, y_train, cv=5, scoring=\'accuracy\')\n    print(f"{name} accuracy: {scores.mean():.3f} +/- {scores.std():.3f}")\n'

In [10]:
gbc= MLPClassifier(max_iter=25, alpha=1e-5, hidden_layer_sizes=(20,), random_state=1)
#param_grid = {'hidden_layer_sizes':range(15,25)}
#gs = GridSearchCV(gbc, param_grid=param_grid,scoring='f1',cv=5)


gbc.fit(x_train, y_train)
y_pred = gbc.predict(x_val)
#print(gs.best_estimator_)
#print("Best score: %0.3f" % gs.best_score_)

#from sklearn.metrics import classification_report
#print(gs.score(x_val,y_val))

acc = accuracy_score(y_val, y_pred)
print (acc)

0.8923568444335217


/opt/conda/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (25) reached and the optimization hasn't converged yet.
  ConvergenceWarning,


In [11]:
scaler.fit(test[num_list])
test[num_list] = scaler.transform(test[num_list])
pred_test = gbc.predict(test)
sub = pd.read_csv("/kaggle/input/ee5180-course-contest/sample_submission.csv")
sub["y"] = pred_test
sub.to_csv("/kaggle/working/submission.csv", index=False)
sub.head()


,id,y
0,5149,0
1,6179,0
2,4846,0
3,3676,0
4,4256,0
